最简单的RNN使用
序列结构有很多种类别：  
* one-to-one，如图像分类
* one-to-many，如图像转文字
* many-to-one，如语义理解判断语句性质
* many-to-many，这个有同步和异步两种
之所以能够RNN进行序列的转换，在于中间的recurrent transformation（循环传输）只要是定长的，那么上下的输入输出就可以是不定长的。


这里的程序是Character-level也就是one-to-one的  
主要分为：变量定义、输入编码、输出解码、构造一个rnn、构造整体模型（softmax）、训练、测试几个部分

In [25]:
import numpy as np
import tensorflow as tf
import os
import time
# 定义常用变量
HIDDEN_SIZE =200#这个是h的尺寸，即W_xh中h的维度
NUM_STEPS = 50 #每个batch中序列的长度
BATCH_SIZE = 64 #batch的数量
LR = 0.003

#这里的可视化我指的是测试过程生成序列
TEMPRATURE = 0.7#同样是可视化时候需要的，进行输出采样
SKIP_STEP = 40 #训练多少次进行一次可视化
LEN_GENERATED = 300#可视化时生成的序列长
log_dir = 'D:/seq2seq/'
DATA_PATH = 'arvix_abstracts.txt'

In [18]:
def vocab_encode(text, vocab):
    #最简单的编码，返回一个list包含所有的text中内容所包含的转换后数据
    return [vocab.index(x)+1 for x in text if x in vocab]
def vocab_decode(array, vocab):
    # array为数字列表
    # 返回类型为string
    return ''.join([vocab[i-1] for i in array])
# 首先看一下数据的读取形式
def read_data(filename, vocab, windows=NUM_STEPS, overlap=NUM_STEPS):
    #按行读取数据
    for text in open(filename):
        text = vocab_encode(text, vocab) #把text中的数据编码为数字
        for start in range(0, len(text)-windows, overlap):
            #把数据分成window长的块
            chunk = text[start: start+windows]
            chunk += [0]*(windows-len(chunk))#这是为了把不够数量的部分用0来pad
            yield chunk
#数据进行batch
def read_batch(stream, batch_size=BATCH_SIZE):
    batch = []
    for element in stream:
        batch.append(element)
        #如果数量batch_size足够就返回
        if len(batch) == batch_size:
            yield batch
            batch = []
    #如果最后达不到batch_size的数量也返回即可
    yield batch

这里用到了几个关键的函数
[tf.nn.rnn_cell.GRUCell]()  
[tf.nn.dynamic_rnn](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

tf.nn.dynamic_rnn的参数：
* out:返回值out维度和seq相同表示的是seq经过rnn的输出值
* out_state：这个就是计算过程中的h(t-1),保存着上一个输出状态，在初始条件下h(0)为None

这里需要提到一个问题，对于不定长的问题我们用了补零的方式进行，但是这样会影响计算的loss的结果，有两种解决方式：
1. 用mask记录哪些是真实值哪些是补零值，然后用模型计算序列得到的结果，但是在计算损失的时候只用计算真实元素的部分
```
full_loss = tf.nn.softmax_cross_entropy_with_logits(preds, labels)
loss = tf.reduce_mean(tf.boolean_mask(full_loss, mask))
```
2. 让模型知道真实序列的长度，然后只计算真实序列的结果
```
cell = tf.nn.rnn_cell.GRUCell(hidden_size)
rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)
length = tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)#计算长度
output, out_state = tf.nn.dynamic_rnn(cell, seq, length, initial_state)#让rnn知道长度
```

In [3]:
def creat_rnn(seq, hidden_size=HIDDEN_SIZE):
    '''
        这里的seq为三维[batch_num,NUM_STEPS,len(vocab)]
    '''
    cell = tf.contrib.rnn.GRUCell(hidden_size) #生成一层的含有hidden_size数量的GRU
    #cell.zero_state返回一个batch_size*数量的tensor
    #这里注意为什么in_state可以定义为placeholder，在哪里用到了
    #不同的batch有不一样的状态，所以共有tf.shape(seq)[0]个不同的in_state
    #cell.zero_state(tf.shape(seq)[0],tf.float32)既可以初始化cell
    in_state = tf.placeholder_with_default(
        cell.zero_state(tf.shape(seq)[0],tf.float32), [None, hidden_size])
    
    #因为不一定所有的seq中的内容都是num_step长的，所以需要进行pad使所有seq长度一样、
    #首先需要计算每个seq的长度，tf.sign让seq中大于0的数变为1(对于本例来说没什么作用)，
    #reduce_max求每一个batch中每个数据最大值（对于pad的数据这个值是0）
    #tf.reduce_sum得到了每个batch中数据的数量
    #生成的length为一个list（tensor）返回每一个batch的长度
    
    length = tf.reduce_sum(tf.reduce_max(tf.sign(seq),2),1)
    
    out, out_state = tf.nn.dynamic_rnn(cell=cell,
                                       inputs=seq,
                                       sequence_length=length, 
                                       initial_state=in_state)
    return out, in_state, out_state

In [4]:
def create_model(seq, temp, vocab, hidden_size= HIDDEN_SIZE):
    '''
        输入： 
            vocab: 可能包含的字典（这个程序是按字符构造的，不是按单词）
            seq: 读取的数据全集，这里seq还是一个二维Tensor
            temp:训练的时候用不到
        输出：
            loss: softmax损失
            sample：采样值，其实不用也可以，只要将
            in_state: 输入状态h(t-1)
            out_state:输出状态h(t)
        
    '''
    seq = tf.one_hot(seq,len(vocab)) #变one-hot，这时的seq是三维的
    output, in_state, out_state = creat_rnn(seq, hidden_size)
    #添加一个fully_connect
    logits = tf.contrib.layers.fully_connected(output, len(vocab),None)
    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=logits[:, :-1],labels=seq[:, 1:]))
    
    # sample the next character from Maxwell-Boltzmann Distribution with temperature temp
    # it works equally well without tf.exp
    #从每一个batch中采样一个结果
    sample = tf.multinomial(tf.exp(logits[:, -1] / temp), 1)[:, 0] 
    return loss, sample, in_state, out_state    

In [5]:
def online_inference(sess, vocab, seq, sample, temp, in_state, out_state, seed='T'):
    """ Generate sequence one character at a time, based on the previous character
        每次生成序列中的一个词， 共LEN_GENERATED次
    """
    sentence = seed
    state = None
    for _ in range(LEN_GENERATED):
        batch = [vocab_encode(sentence[-1], vocab)]
        feed = {seq: batch, temp: TEMPRATURE}
        # for the first decoder step, the state is None
        # 开始的时候in_state是None
        if state is not None:
            feed.update({in_state: state})
        #index是输出字符，state是新的状态值
        index, state = sess.run([sample, out_state], feed)
        sentence += vocab_decode(index, vocab)
    print(sentence)

利用上面的函数构建图

In [6]:
vocab = (
        " $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        "\\^_abcdefghijklmnopqrstuvwxyz{|}")
#这时的seq还是二维的
seq = tf.placeholder(tf.int32,[None,None])
temp = tf.placeholder(tf.float32)
loss, sample, in_state, out_state = create_model(seq, temp, vocab)
global_step = tf.Variable(0,dtype=tf.int32, trainable=False, name='global_step')
optimizer = tf.train.AdamOptimizer(LR).minimize(loss, global_step=global_step)

现在开始进行训练过程

In [ ]:
saver = tf.train.Saver()
start = time.time()
with tf.Session() as sess:
    writer = tf.summary.FileWriter('gist',sess.graph)
    sess.run(tf.global_variables_initializer())
    ckpt = tf.train.get_checkpoint_state(os.path.dirname(log_dir+ 'checkpoint'))
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    iteration = global_step.eval()
    for batch in read_batch(read_data(DATA_PATH, vocab)):
        batch_loss,_ = sess.run([loss, optimizer], {seq:batch})
        #下面是为了可视化,每SKIP_STEP对结果进行一次输出
        if (iteration+1)%SKIP_STEP == 0:
            print('Iter {}. \n    Loss {}. Time {}'.format(iteration, batch_loss, time.time() - start))
            online_inference(sess, vocab, seq, sample, temp, in_state, out_state)
            start = time.time()
            saver.save(sess, log_dir+'checkpoints', iteration)
        iteration+=1